In [14]:
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesRegressor

# Set random seed
np.random.seed(0)
# the dir of train.csv and test.csv
workdic='./'
df = pd.read_csv(workdic + "train.csv")
y = df['time']
def times(a, b):
    return a * b
def change_clo(df):
  mask = df.n_jobs <0 
  mask1 = df.n_jobs >9
  column_name = 'n_jobs'
  df.loc[mask, column_name] = 9
  df.loc[mask1, column_name] = 9
  return df
df = change_clo(df)
df['data_num'] = df.apply(lambda row: times(row['n_samples'], row['n_features']), axis=1)
df['max_scan'] = df.apply(lambda row: times(row['data_num'], row['max_iter']), axis=1)
df['max_scan_alpha'] = df.apply(lambda row: times(row['max_scan'], row['alpha']), axis=1)
df['max_scan_n_jobs'] = df.apply(lambda row: times(row['max_scan'], 1/row['n_jobs']), axis=1)
df['max_scan_n_classes'] = df.apply(lambda row: times(row['max_scan'], row['n_classes']), axis=1)
df['max_scan_n_classes_job'] = df.apply(lambda row: times(row['max_scan_n_classes'], 1/row['n_jobs']), axis=1)
df['data_num_job'] = df.apply(lambda row: times(row['data_num'], 1/row['n_jobs']), axis=1)
df['max_time'] = np.max(df['time'])
df['min_time'] = np.min(df['time'])
df['mean_time'] = np.mean(df['time'])
df['median_time'] = np.median(df['time'])
df['sum_time'] = np.sum(df['time'])
df['var_time'] = np.var(df['time'])
df['std_time'] = np.std(df['time'])
tmp_df = df
test_df = pd.read_csv(workdic + "test.csv")
test_df = change_clo(test_df)
test_df['data_num'] = test_df.apply(lambda row: times(row['n_samples'], row['n_features']), axis=1)
test_df['max_scan'] = test_df.apply(lambda row: times(row['data_num'], row['max_iter']), axis=1)
test_df['max_scan_alpha'] = test_df.apply(lambda row: times(row['max_scan'], row['alpha']), axis=1)
test_df['max_scan_n_jobs'] = test_df.apply(lambda row: times(row['max_scan'], 1/row['n_jobs']), axis=1)
test_df['max_scan_n_classes'] = test_df.apply(lambda row: times(row['max_scan'], row['n_classes']), axis=1)
test_df['max_scan_n_classes_job'] = test_df.apply(lambda row: times(row['max_scan_n_classes'], 1/row['n_jobs']), axis=1)
test_df['data_num_job'] = test_df.apply(lambda row: times(row['data_num'], 1/row['n_jobs']), axis=1)
test_df['max_time'] = np.max(df['time'])
test_df['min_time'] = np.min(df['time'])
test_df['mean_time'] = np.mean(df['time'])
test_df['median_time'] = np.median(df['time'])
test_df['sum_time'] = np.sum(df['time'])
test_df['var_time'] = np.var(df['time'])
test_df['std_time'] = np.std(df['time'])
del tmp_df['id']
del tmp_df['time']
x = pd.get_dummies(tmp_df)
test_id=test_df['id']
del test_df['id']
output_df = pd.DataFrame(columns = ['Id','time'])
output_df['Id'] = test_id
X_test = pd.get_dummies(test_df)
features = x.columns[:len(x.columns)]
test_features = X_test.columns[:len(X_test.columns)]
for i in features:
    if i not in test_features:
        print(i)
        X_test[i] = 0


In [17]:
'''
best                          
1.66171,                          

etr = ExtraTreesRegressor(n_estimators=3000,random_state=0, max_depth=50, 
                          bootstrap=False, n_jobs=-1, min_samples_split=3,
'''

etr = ExtraTreesRegressor(n_estimators=3000,random_state=0, max_depth=50, 
                          bootstrap=False, n_jobs=-1, min_samples_split=3)
predicted = cross_val_predict(etr, x, y, cv=4)
mse = mean_squared_error(y, predicted)
print('mse:',mse)
etr.fit(x,y)


mse: 2.4325566398056266


ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=50,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=3,
          min_weight_fraction_leaf=0.0, n_estimators=3000, n_jobs=-1,
          oob_score=False, random_state=0, verbose=0, warm_start=False)

In [19]:
predi_y = etr.predict(X_test)
test_df['time'] = predi_y
output_df['time'] = predi_y
output_df.to_csv('etr_Submission_1122.csv', index=False)